In [1]:
# 1. INSTALACIÓN Y CARGA DE DEPENDENCIAS
!pip install transformers sentence-transformers accelerate bitsandbytes numpy pandas pyarrow datasets torch faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 29.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.2 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━

In [6]:
# ==============================================================================
# 1. DEPENDENCIAS Y LIBRERÍAS
# ==============================================================================
import pandas as pd
import numpy as np
import torch
import faiss
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import warnings
warnings.filterwarnings("ignore") # Ignorar warnings de tokenizers y modelos


# ==============================================================================
# 2. CARGA Y PROCESAMIENTO DEL CORPUS (Usando solo la Cover Letter)
# ==============================================================================
def cargar_corpus():
    print("--- 1. Cargando Dataset y Corpus ---")
    try:
        ds = load_dataset("dhruvvaidh/cover-letter-dataset-llama3")
        train_df = ds["train"].to_pandas()
    except Exception as e:
        print(f"Error cargando dataset: {e}. Usando datos dummy para demostración.")
        train_df = pd.DataFrame({
            "Instruction": ["I1", "I2"],
            "Prompt": ["Job: Python Dev. Candidate: Expert in Django...", "Job: Data Scientist. Candidate: Expert in Pandas..."],
            "Output": ["Dear Hiring Manager, my skills in Python and Django make me a perfect fit for this role...", "To Whom It May Concern, I have a strong background in data science and deep learning..."]
        })

    def canonize_row(r):
        output = str(r.get("Output", "")).strip()
        return {
            "doc_id": r.name,
            "text_for_rag": output,
        }

    corpus_df = train_df.apply(canonize_row, axis=1, result_type="expand")
    corpus_list = corpus_df.to_dict('records')
    textos_para_indexar = corpus_df['text_for_rag'].tolist()
    
    print(f"Corpus cargado: {len(corpus_list)} documentos.")
    return corpus_list, textos_para_indexar

corpus_list, textos_para_indexar = cargar_corpus()

--- 1. Cargando Dataset y Corpus ---
Corpus cargado: 813 documentos.


In [7]:
# ==============================================================================
# 3. INDEXACIÓN (BGE + FAISS CPU) 
# ==============================================================================
print("\n--- 2. Generando Embeddings e Índice FAISS (BGE) ---")
EMBED_MODEL_ID = "BAAI/bge-large-en-v1.5"
try:
    embedder = SentenceTransformer(EMBED_MODEL_ID)
    doc_embeddings = embedder.encode(
        textos_para_indexar, 
        batch_size=32, 
        show_progress_bar=True, 
        normalize_embeddings=True
    )
    doc_embeddings = np.array(doc_embeddings, dtype="float32")
    d_dimension = doc_embeddings.shape[1]

    index = faiss.IndexFlatIP(d_dimension)
    index.add(doc_embeddings)
    print(f"Índice FAISS creado en CPU con {index.ntotal} vectores.")

except Exception as e:
    print(f"Error crítico en embeddings/FAISS. {e}")
    embedder = None
    index = None

# ==============================================================================
# 4. FUNCIÓN DE BÚSQUEDA (RETRIEVER) 
# ==============================================================================
def buscar_candidatos(query, k=3):
    if index is None or embedder is None:
        return []
    
    q_text = "Represent this sentence for searching relevant passages: " + query
    
    q_emb = embedder.encode([q_text], normalize_embeddings=True)
    q_emb = np.array(q_emb, dtype="float32")
    
    scores, indices = index.search(q_emb, k)
    
    resultados = []
    for idx, score in zip(indices[0], scores[0]):
        if idx != -1:
            doc_data = corpus_list[idx]
            resultados.append({
                "id": doc_data['doc_id'],
                "score": float(score),
                "context": doc_data['text_for_rag'], # Cover Letter completa
            })
    return resultados


--- 2. Generando Embeddings e Índice FAISS (BGE) ---


Batches:   0%|          | 0/26 [00:00<?, ?it/s]

Índice FAISS creado en CPU con 813 vectores.


In [ ]:
# ==============================================================================
# 5. CONFIGURACIÓN DEL GENERATOR (LLM para Ranking) 
# ==============================================================================
LLM_ID = "Qwen/Qwen2.5-3B"
print(f"\n--- 3. Cargando LLM Generador: {LLM_ID} (Contexto 128K tokens) ---")

try:
    # Configuración de 4 bits para ahorrar memoria
    quantization_config = BitsAndBytesConfig(load_in_4bit=True)

    tokenizer = AutoTokenizer.from_pretrained(LLM_ID)
    llm_model = AutoModelForCausalLM.from_pretrained(
        LLM_ID,
        quantization_config=quantization_config,
        device_map="auto"
    )
    
    # Configuración del tokenizer para generación causal
    tokenizer.pad_token = tokenizer.eos_token 
    tokenizer.padding_side = "left" 

    ranker_pipeline = pipeline(
        "text-generation", 
        model=llm_model,
        tokenizer=tokenizer,
        device_map="auto"
    )
    
    print("LLM cargado correctamente en memoria reducida (4-bit).")
except Exception as e:
    print(f"Error cargando LLM o libs de cuantización (bitsandbytes/accelerate). {e}")
    ranker_pipeline = None

In [18]:
def generar_ranking_llm(job_offer, candidatos):
    """Genera un ranking usando Qwen2.5-3B usando la plantilla de chat correcta."""
    
    global ranker_pipeline
    if not ranker_pipeline:
        return "Error: LLM no disponible."
    
    # 1. Construir el texto de los candidatos
    contexto_str = ""
    for i, c in enumerate(candidatos, 1):
        perfil = c['context'].replace("\n", " ").strip()
        contexto_str += f"CANDIDATE {i} (ID {c['id']}) - COVER LETTER: {perfil}\n\n"
    
    
    # 2. Definición de mensajes con Prompt Engineering avanzado
        messages = [
            {
                "role": "system",
                "content": (
                    "You are an expert  Technical Recruiter. Your job is to evaluate candidates based strictly "
                    "on how well their application matches the provided Job Description. "
                    "You must provide critical reasoning, not just a summary and explain your decission in 3-5 lines."
                )
            },
            {
                "role": "user",
                "content": f"""
    ### JOB DESCRIPTION:
    {job_offer}
    
    ### CANDIDATES TO EVALUATE:
    {contexto_str}
    
    ### YOUR TASK:
    Rank the candidates from Best Fit (1) to Worst Fit.
    
    ### REQUIRED OUTPUT FORMAT:
    For each candidate, you must use exactly this format:
    
    1. **Candidate ID [ID]**: [Best/Good/Weak] Match
       - **Reasoning**: [Explain specifically WHY. Mention specific skills from their letter that match the job (e.g., "Has 5 years in Python", "Mentions AWS"). Mention missing skills if any.]
    
    2. **Candidate ID [ID]**: ...
    
    (Do NOT copy the full cover letter. Only provide the ranking and the reasoning).
    """
            }
        ]
    
        # 3. Aplicar plantilla de chat
        prompt = ranker_pipeline.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        # 4. Generación
        outputs = ranker_pipeline(
            prompt, 
            max_new_tokens=500,  # Damos espacio para que se explique
            temperature=0.2,     # Temperatura baja para ser más analítico y menos creativo
            do_sample=True,
            return_full_text=False
        )
        
        return outputs[0]['generated_text'].strip()

In [19]:
# ==============================================================================
# 7. FUNCIÓN RAG COMPLETA AUTOMATIZADA 
# ==============================================================================
def ejecutar_rag_pipeline(job_offer_query, k=3):
    """
    Ejecuta el pipeline RAG completo para encontrar y rankear candidatos.

    Args:
        job_offer_query (str): La nueva oferta de trabajo.
        k (int): Número de candidatos a recuperar (Top-K).

    Returns:
        dict: Contiene el ranking generado por el LLM.
    """
    print(f"\n{'='*60}")
    print(f"INICIANDO RAG para: {job_offer_query}")
    print(f"Buscando los {k} mejores candidatos (usando Cover Letters completas)...")
    print(f"{'='*60}")

    # 1. RECUPERACIÓN (Retrieval - BGE + FAISS)
    candidatos_encontrados = buscar_candidatos(job_offer_query, k=k)

    if not candidatos_encontrados:
        print("\n No se encontraron candidatos relevantes. Terminando el pipeline.")
        return { "ranking_final_llm": "No se encontraron candidatos para rankear." }

    print("\n[FASE 1: RECUPERACIÓN COMPLETADA]")
    for i, c in enumerate(candidatos_encontrados, 1):
        print(f"  {i}. Candidato ID: {c['id']} | Similitud BGE: {c['score']:.4f}")

    # 2. RANKING/GENERACIÓN (Generation - Qwen2.5-3B)
    print(f"\n{'-'*60}")
    print("INICIANDO FASE DE RANKING (LLM)...")
    
    ranking_generado = generar_ranking_llm(job_offer_query, candidatos_encontrados)
    
    print(f"{'-'*60}")
    print("REPORTE FINAL DE RR.HH:")
    print(ranking_generado)

    return { "ranking_final_llm": ranking_generado }

# ==============================================================================
# 8. EJEMPLO DE USO
# ==============================================================================
TARGET_JOB = "We need a Project Manager with PMP certification, strong leadership, experience in transformer-based models, Python, and cloud"
ejecutar_rag_pipeline(TARGET_JOB, k=3)


INICIANDO RAG para: We need a Project Manager with PMP certification, strong leadership, experience in transformer-based models, Python, and cloud
Buscando los 3 mejores candidatos (usando Cover Letters completas)...

[FASE 1: RECUPERACIÓN COMPLETADA]
  1. Candidato ID: 811 | Similitud BGE: 0.6717
  2. Candidato ID: 712 | Similitud BGE: 0.6603
  3. Candidato ID: 214 | Similitud BGE: 0.6597

------------------------------------------------------------
INICIANDO FASE DE RANKING (LLM)...
------------------------------------------------------------
REPORTE FINAL DE RR.HH:
Candidate 1 (ID 811): Best Match
Reasoning: Candidate 811 has a strong background in data analysis and project management, which aligns with the requirements of the job description. Additionally, their experience in transformer-based models, Python, and cloud aligns with the job requirements. Their cover letter also mentions their PMP certification, which is a strong indicator of their project management skills. They als

{'ranking_final_llm': 'Candidate 1 (ID 811): Best Match\nReasoning: Candidate 811 has a strong background in data analysis and project management, which aligns with the requirements of the job description. Additionally, their experience in transformer-based models, Python, and cloud aligns with the job requirements. Their cover letter also mentions their PMP certification, which is a strong indicator of their project management skills. They also mention their collaboration with cross-functional teams, which is a key aspect of the job. Their communication and problem-solving skills are also mentioned, which are important for the job. Overall, their cover letter shows that they have the necessary skills and experience for the job.'}